In [1]:
#using pandas
import os
import pandas as pd
import numpy as np
from numpy import *


In [2]:
#!type marketplan.csv

In [3]:
data = genfromtxt('marketplan.csv',skip_header=1, delimiter=',', dtype=None,usecols=arange(0,2))  

In [4]:
x=data

In [5]:
a=np.array(x)
a

array([['Newyork', 'WIRELESS'],
       ['Newyork', 'WIRELESS'],
       ['Newyork', 'WIRELESS'],
       ..., 
       ['Missouri', 'CB-WIRELINE'],
       ['Missouri', 'CB-WIRELINE'],
       ['Missouri', 'CB-WIRELINE']],
      dtype='|S33')

# Analyzing ATT data sets with Frequent pattern growth algorithm

In [6]:
#variables:
#name of the node, a count
#nodelink used to link similar items
#parent vaiable used to refer to the parent of the node in the tree
#node contains an empty dictionary for the children in the node
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [7]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [8]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [9]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [10]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [11]:
initSet = createInitSet(a)
initSet

{frozenset({'Albama', 'WIRELESS'}): 1,
 frozenset({'CB-WIRELINE', 'Newyork'}): 1,
 frozenset({'CB-WIRELINE', 'Ohio'}): 1,
 frozenset({'Newyork', 'WIRELESS'}): 1,
 frozenset({'Penselvaniya', 'WIRELESS'}): 1,
 frozenset({'Albama', 'Unified (Uverse + Wireless + DTV)'}): 1,
 frozenset({'CB-WIRELINE', 'Penselvaniya'}): 1,
 frozenset({'California', 'WIRELESS'}): 1,
 frozenset({'CB-U-VERSE', 'Texas'}): 1,
 frozenset({'Newyork', 'Unified (Uverse + Wireless + DTV)'}): 1,
 frozenset({'CB-WIRELINE', 'Missouri'}): 1,
 frozenset({'California', 'Unified (Uverse + Wireless + DTV)'}): 1,
 frozenset({'Indiana', 'Unified (Uverse + Wireless + DTV)'}): 1,
 frozenset({'Albama', 'CB-U-VERSE'}): 1,
 frozenset({'Ohio', 'WIRELESS'}): 1,
 frozenset({'Texas', 'WIRELESS'}): 1,
 frozenset({'CB-U-VERSE', 'Newyork'}): 1}

In [12]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

('  ', 'Null Set', ' ', 1)
('    ', 'Unified (Uverse + Wireless + DTV)', ' ', 3)
('      ', 'Albama', ' ', 1)
('    ', 'Newyork', ' ', 3)
('      ', 'CB-WIRELINE', ' ', 1)
('      ', 'Unified (Uverse + Wireless + DTV)', ' ', 1)
('      ', 'CB-U-VERSE', ' ', 1)
('    ', 'CB-U-VERSE', ' ', 1)
('    ', 'WIRELESS', ' ', 6)
('      ', 'Newyork', ' ', 1)
('      ', 'Albama', ' ', 1)
('    ', 'CB-WIRELINE', ' ', 3)
('    ', 'Albama', ' ', 1)
('      ', 'CB-U-VERSE', ' ', 1)


# Mining frequent items from an FP-tree

In [13]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [14]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [15]:
findPrefixPath('Newyork', myHeaderTab['Newyork'][1])

{frozenset({'WIRELESS'}): 1}

In [16]:
findPrefixPath('Albama', myHeaderTab['Albama'][1])

{frozenset({'Unified (Uverse + Wireless + DTV)'}): 1,
 frozenset({'WIRELESS'}): 1}